In [2]:
import os
import pandas as pd


def fixpath(path):
    if path.startswith("C:"): return "/mnt/c/" + path.replace("\\", "/")[3:]
    else:
        pass
    return path

silver_path = fixpath(r'C:\SILVER_BC_Cascade\SILVER_Data\user_inputs\Hydro_Data-BC_Cascade')
os.chdir(silver_path)

# Read the hydro data

cascade_pmin_sf = pd.read_csv('hydro_cascade_pmin - saf.csv', index_col=0)
cascade_pmin_tk = pd.read_csv('hydro_cascade_pmin - TK.csv', index_col=0)

max_cap_cas = pd.read_excel(f'{os.path.dirname(silver_path)}/model inputs - BC_cascade_scen14.xlsx', sheet_name='non-vre plants')
max_cap_cas = max_cap_cas.loc[max_cap_cas['name'].str.contains('cascade'), ['name', '[MW]']].transpose()
max_cap_cas_h = pd.concat([max_cap_cas]*len(cascade_pmin_sf.index), ignore_index=True)
max_cap_cas_h = max_cap_cas_h.loc[pd.to_numeric(max_cap_cas_h.iloc[:,1], errors='coerce').notnull()]
max_cap_cas_h.index = cascade_pmin_sf.index
max_cap_cas_h.columns = cascade_pmin_sf.columns[0:25]

hourly_pmin_sf = pd.read_csv('hydro_hourly_pmin - saf.csv', index_col=0)
hourly_pmin_tk = pd.read_csv('hydro_hourly_pmin - TK.csv', index_col=0)

In [33]:
# Select the periods of interest

periods = input("Enter the period of interest in dates (e.g. 2050-01-01 to 2050-03-01): ")
periods = periods.split('to')
periods = [pd.to_datetime(periods[0]), pd.to_datetime(periods[1])]
timeline = pd.date_range(periods[0], periods[1], freq='H')
print(timeline)

DatetimeIndex(['2050-01-01 00:00:00', '2050-01-01 01:00:00',
               '2050-01-01 02:00:00', '2050-01-01 03:00:00',
               '2050-01-01 04:00:00', '2050-01-01 05:00:00',
               '2050-01-01 06:00:00', '2050-01-01 07:00:00',
               '2050-01-01 08:00:00', '2050-01-01 09:00:00',
               '2050-01-01 10:00:00', '2050-01-01 11:00:00',
               '2050-01-01 12:00:00', '2050-01-01 13:00:00',
               '2050-01-01 14:00:00', '2050-01-01 15:00:00',
               '2050-01-01 16:00:00', '2050-01-01 17:00:00',
               '2050-01-01 18:00:00', '2050-01-01 19:00:00',
               '2050-01-01 20:00:00', '2050-01-01 21:00:00',
               '2050-01-01 22:00:00', '2050-01-01 23:00:00',
               '2050-01-02 00:00:00'],
              dtype='datetime64[ns]', freq='H')


In [45]:
# Selecting the cascades of interest

print("The following cascades are available: ")
print(cascade_cf.columns[1:-3])
imp_cascades = input("Enter the cascade(s) of interest from the list (e.g. 1,2,3): ")
imp_cascades = imp_cascades.split(',')
imp_cascades = [int(i) for i in imp_cascades]
imp_cascades = [cascade_cf.columns[i] for i in imp_cascades]
print(imp_cascades)

The following cascades are available: 
Index(['cascade_1', 'cascade_2', 'cascade_3', 'cascade_4', 'cascade_5',
       'cascade_6', 'cascade_7', 'cascade_8', 'cascade_9', 'cascade_10',
       'cascade_11', 'cascade_12', 'cascade_13', 'cascade_14', 'cascade_15',
       'cascade_16', 'cascade_17', 'cascade_18', 'cascade_19', 'cascade_20',
       'cascade_21', 'cascade_22', 'cascade_23', 'cascade_24', 'cascade_25'],
      dtype='object')
['cascade_2', 'cascade_4']


In [63]:
# Changing the minimum capacity of the cascades

imp_factor = input("Enter the factor by which the cascading minimum caoacity should be increased (e.g. 1.2): ")
imp_factor = float(imp_factor)
imp_cascade_pmin = pd.DataFrame(cascade_pmin)
imp_cascade_pmin.loc[(cascade_pmin.index >= timeline[0]) & (cascade_pmin.index <= timeline[-1]), imp_cascades] = cascade_pmin.loc[(cascade_pmin.index >= timeline[0]) & (cascade_pmin.index <= timeline[-1]), imp_cascades] * imp_factor

In [59]:
# Writing to new files

scen = input("Enter the scenario name: ")
imp_cascade_pmin.to_csv('hydro_cascade_pmin_' + scen + '.csv', index=False)

In [4]:
# Rabdnomly producing a range for the hourly minimum capacity

from scipy.stats.qmc import LatinHypercube as LH

# Create a Latin Hypercube sampler
engine = LH(1)

cascade_range = 0.55*max_cap_cas_h - cascade_pmin_sf
hourly_range = hourly_pmin_tk - hourly_pmin_sf

rand_cascade = pd.DataFrame(engine.random(10))
rand_cascade.sort_values(by=0, inplace=True)
rand_cascade.reset_index(drop=True, inplace=True)
rand_hourly = pd.DataFrame(engine.random(10))
rand_hourly.sort_values(by=0, inplace=True)
rand_hourly.reset_index(drop=True, inplace=True)

# Select random capacities to be sampled
rand_cascade_cap = cascade_pmin_sf.sample(n=int(round(len(cascade_pmin_sf.columns)*0.5,0)), axis=1)
rand_cascade_cap.dropna(axis=1, inplace=True)
rand_hourly_cap = hourly_pmin_sf.sample(n=int(round(len(hourly_pmin_sf.columns)*0.5,0)), axis=1)
rand_hourly_cap.dropna(axis=1, inplace=True)

In [23]:

# Create a random sample
for i in range(0,10):
    locals()[f'cascade_sample_{i}'] = round(rand_cascade.at[i,0] * cascade_range + cascade_pmin_sf,3)
    locals()[f'hourly_sample_{i}'] = round(rand_hourly.at[i,0] * hourly_range + hourly_pmin_sf,3)

# Write random pmins to csvs
for i in range(0,10):
    locals()[f'cascade_sample_{i}'].loc[:,rand_cascade_cap.columns] = rand_cascade_cap
    locals()[f'cascade_sample_{i}'].to_csv(f'hydro_cascade_pmin_{i}.csv')
    locals()[f'hourly_sample_{i}'].loc[:,rand_hourly_cap.columns] = rand_hourly_cap
    locals()[f'hourly_sample_{i}'].to_csv(f'hydro_hourly_pmin_{i}.csv')